In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from scipy import stats
import numpy as np
from datetime import datetime, timedelta
from scipy.stats import zscore,ttest_ind

E:\anaconda3\envs\tf\lib\site-packages\scipy\__init__.py:173: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [18]:
file_loc0 = "../../ADNI_data/"
df_FDG=pd.read_csv(file_loc0 + 'UCBERKELEYFDG_8mm_02_17_23_23Apr2024.csv')
df_Fpet_npiq_DX_62days_final3=pd.read_csv('df_Fpet_npiq_VISDATE_DX_182days_final3.csv')


df_NPIQ=pd.read_csv(file_loc0 + 'NPIQ_05Mar2024.csv')
df_DX = pd.read_csv(file_loc0 + 'DXSUM_PDXCONV_ADNIALL_Diagnostic Summary_standardised.csv')

df_Fpet_npiq_DX_62days_final3.shape,df_FDG.shape

((962, 17), (7524, 10))

In [25]:
df_FDG.head()

,RID,VISCODE,VISCODE2,EXAMDATE,ROINAME,MEAN,MAX,STDEV,TOTVOX,update_stamp,EXAMDATE_dt
0,2,bl,bl,2005-09-22,MetaROI,1.252,1.672,0.159,1142.0,2023-02-23 10:07:26.0,2005-09-22
1,2,bl,bl,2005-09-22,Top50PonsVermis,0.937,1.297,0.107,1779.0,2023-02-23 10:07:26.0,2005-09-22
2,2,m06,m06,2006-03-08,MetaROI,1.333,1.644,0.166,1142.0,2023-02-23 10:07:26.0,2006-03-08
3,2,m06,m06,2006-03-08,Top50PonsVermis,1.090,1.554,0.170,1779.0,2023-02-23 10:07:26.0,2006-03-08
4,3,bl,bl,2005-09-22,MetaROI,1.222,1.570,0.137,1142.0,2023-02-23 10:07:26.0,2005-09-22


In [4]:
df_Fpet_npiq_DX_62days_final3.head()

,RID,VISCODE2,USERDATE_dt,VISDATE_dt,NPID,SCANDATE_dt,AMYLOID_STATUS,SUMMARY_SUVR,diff_dt,diff_dt_abs,VISCODE2_dx,EXAMDATE_dt,DX,diff_dx_dt,diff_dx_dt_abs,diff_ED_VD_dt,diff_ud-vd
0,4185,m18,2013-04-05,2013-04-05,0.0,2013-09-30,0.0,1.044,-178 days,178 days,m24,2013-10-03,MCI,3 days,3 days,-181 days,0 days
1,4635,m18,2013-11-05,2013-10-31,0.0,2014-04-29,1.0,1.362,-180 days,180 days,m24,2014-04-30,MCI,1 days,1 days,-181 days,5 days
2,4225,m18,2013-05-14,2013-04-12,0.0,2013-10-10,1.0,1.642,-181 days,181 days,m24,2013-10-10,NL,0 days,0 days,-181 days,32 days
3,1261,m66,2012-09-11,2012-09-07,0.0,2013-03-05,0.0,1.028,-179 days,179 days,m72,2013-03-07,NL,2 days,2 days,-181 days,4 days
4,4598,m18,2013-10-24,2013-10-22,0.0,2014-04-21,0.0,1.050,-181 days,181 days,m24,2014-04-21,NL,0 days,0 days,-181 days,2 days


In [26]:
df_Fpet_npiq_DX_62days_final3.merge(df_FDG,on=['RID','VISCODE2']).shape

(496, 26)

In [28]:
df_FDG[['RID','VISCODE2']].shape, df_FDG[['RID','VISCODE2']].drop_duplicates().shape, df_FDG[['RID','VISCODE2']].drop_duplicates().dropna().shape

((7524, 2), (3762, 2), (3760, 2))

In [7]:
3762*2

7524

In [29]:
df_FDG_ROI=df_FDG.pivot_table(index=['RID','VISCODE2'],columns='ROINAME',values='MEAN').dropna().reset_index()
df_FDG_ROI.shape,df_FDG_ROI.columns

((3758, 4),
 Index(['RID', 'VISCODE2', 'MetaROI', 'Top50PonsVermis'], dtype='object', name='ROINAME'))

In [9]:
3758*2

7516

In [10]:
df_FDG_ROI[['RID','VISCODE2']].drop_duplicates().shape

(3758, 2)

In [30]:
df_FDG_ROI['ROInorm']=df_FDG_ROI['MetaROI']/df_FDG_ROI['Top50PonsVermis']

In [31]:
df_FDG_ROI.head()

ROINAME,RID,VISCODE2,MetaROI,Top50PonsVermis,ROInorm
0,2,bl,1.252,0.937,1.336179
1,2,m06,1.333,1.090,1.222936
2,3,bl,1.222,1.102,1.108893
3,3,m06,1.195,1.109,1.077547
4,3,m12,1.186,1.062,1.116761


In [12]:
df_Fpet_npiq_DX_62days_final3_FDG_ROI=df_Fpet_npiq_DX_62days_final3.merge(df_FDG_ROI,on=['RID','VISCODE2'])
df_Fpet_npiq_DX_62days_final3_FDG_ROI.shape

(248, 20)

In [13]:
df_Fpet_npiq_DX_62days_final3_FDG_ROI.groupby(['DX','NPID'])['ROInorm'].count()

DX         NPID
AD         0.0       9
           1.0      14
AD to MCI  0.0       1
MCI        0.0     120
           1.0      40
MCI to AD  0.0       5
           1.0       3
MCI to NL  0.0       2
NL         0.0      45
           1.0       3
NL to MCI  0.0       5
           1.0       1
Name: ROInorm, dtype: int64

In [14]:
df_Fpet_npiq_DX_62days_final3_FDG_ROI.loc[df_Fpet_npiq_DX_62days_final3_FDG_ROI['DX']=='MCI to AD','DX']='AD'
df_Fpet_npiq_DX_62days_final3_FDG_ROI.loc[df_Fpet_npiq_DX_62days_final3_FDG_ROI['DX']=='NL to MCI','DX']='MCI'

In [15]:
df_Fpet_npiq_DX_62days_final3_FDG_ROI[df_Fpet_npiq_DX_62days_final3_FDG_ROI['DX'].isin(['NL','MCI','AD'])].groupby(['DX','NPID'])['ROInorm'].count()

DX   NPID
AD   0.0      14
     1.0      17
MCI  0.0     125
     1.0      41
NL   0.0      45
     1.0       3
Name: ROInorm, dtype: int64

In [16]:
df_Fpet_npiq_DX_62days_final3_FDG_ROI[df_Fpet_npiq_DX_62days_final3_FDG_ROI['DX'].isin(['NL','MCI','AD'])].groupby(['DX','NPID'])['ROInorm'].mean()

DX   NPID
AD   0.0     1.016800
     1.0     1.011881
MCI  0.0     1.244109
     1.0     1.254374
NL   0.0     1.247378
     1.0     1.184485
Name: ROInorm, dtype: float64

In [24]:
#convert to data time for manipulation
df_FDG['EXAMDATE_dt']=pd.to_datetime(df_Fpet.EXAMDATE, format='%Y-%m-%d')

df_NPIQ['VISDATE_dt']=pd.to_datetime(df_NPIQ.VISDATE, format='%Y-%m-%d')
df_NPIQ['USERDATE_dt']=pd.to_datetime(df_NPIQ.USERDATE, format='%Y-%m-%d')

df_DX['EXAMDATE_dt']=pd.to_datetime(df_DX.EXAMDATE, format='%m/%d/%Y')

In [21]:
df_NPIQ_selcol_lt=['RID','Phase','VISCODE2','NPID','VISDATE','VISDATE_dt','USERDATE','USERDATE_dt']

In [32]:
df_NPIQ_selcol = df_NPIQ[df_NPIQ_selcol_lt]
df_NPIQ_selcol.head()

,RID,Phase,VISCODE2,NPID,VISDATE,VISDATE_dt,USERDATE,USERDATE_dt
0,2,ADNI1,bl,0.0,2005-09-08,2005-09-08,2005-09-09,2005-09-09
1,3,ADNI1,bl,0.0,2005-09-12,2005-09-12,2005-09-13,2005-09-13
2,5,ADNI1,bl,0.0,2005-09-07,2005-09-07,2005-09-14,2005-09-14
3,8,ADNI1,bl,0.0,2005-09-19,2005-09-19,2005-09-26,2005-09-26
4,7,ADNI1,bl,1.0,2005-10-06,2005-10-06,2005-10-13,2005-10-13


In [35]:
#minimum date to start from w.r.t scan date
# df_NPIQ[df_NPIQ['VISDATE_dt']>(df_Fpet_selcol.SCANDATE_dt.min()-timedelta(days=182))].shape,df_NPIQ.shape

In [36]:
df_FDG_ROI_NPIQ=df_FDG_ROI.merge(df_NPIQ_selcol,on=['RID','VISCODE2'])
df_FDG_ROI_NPIQ.shape

(2012, 11)

In [37]:
df_FDG_ROI_NPIQ.head(2)

,RID,VISCODE2,MetaROI,Top50PonsVermis,ROInorm,Phase,NPID,VISDATE,VISDATE_dt,USERDATE,USERDATE_dt
0,2,bl,1.252,0.937,1.336179,ADNI1,0.0,2005-09-08,2005-09-08,2005-09-09,2005-09-09
1,2,m06,1.333,1.090,1.222936,ADNI1,0.0,2006-03-06,2006-03-06,2006-03-07,2006-03-07


In [40]:
df_FDG_ROI_NPIQ.NPID.value_counts()

 0.0    1536
 1.0     463
-1.0       6
 2.0       6
Name: NPID, dtype: int64

In [41]:
df_FDG_ROI_NPIQ[df_FDG_ROI_NPIQ['NPID'].isin([0,1])].groupby('NPID').ROInorm.mean()

NPID
0.0    1.170693
1.0    1.110058
Name: ROInorm, dtype: float64

In [47]:
df_DX_adapt = df_DX[['RID','VISCODE2','EXAMDATE_dt','DX']].drop_duplicates().sort_values(by=['RID','EXAMDATE_dt'])
df_DX_adapt.shape

(13491, 4)

In [51]:
df_DX_adapt.head()

,RID,VISCODE2,EXAMDATE_dt,DX
0,2,bl,2005-09-29,NL
166,2,m06,2006-03-06,NL
2861,2,m36,2008-08-27,NL
3907,2,m60,2010-09-27,NL
4701,2,m72,2011-09-19,NL


In [50]:
df_FDG_ROI_npiq_DX=df_FDG_ROI_NPIQ.merge(df_DX_adapt,on=['RID','VISCODE2'])
df_FDG_ROI_npiq_DX.shape,df_FDG_ROI_NPIQ.shape

((2009, 13), (2012, 11))

In [52]:
df_FDG_ROI_npiq_DX.head()

,RID,VISCODE2,MetaROI,Top50PonsVermis,ROInorm,Phase,NPID,VISDATE,VISDATE_dt,USERDATE,USERDATE_dt,EXAMDATE_dt,DX
0,2,bl,1.252,0.937,1.336179,ADNI1,0.0,2005-09-08,2005-09-08,2005-09-09,2005-09-09,2005-09-29,NL
1,2,m06,1.333,1.090,1.222936,ADNI1,0.0,2006-03-06,2006-03-06,2006-03-07,2006-03-07,2006-03-06,NL
2,3,bl,1.222,1.102,1.108893,ADNI1,0.0,2005-09-12,2005-09-12,2005-09-13,2005-09-13,2005-09-30,AD
3,3,m06,1.195,1.109,1.077547,ADNI1,0.0,2006-03-13,2006-03-13,2006-03-13,2006-03-13,2006-03-13,AD
4,3,m12,1.186,1.062,1.116761,ADNI1,0.0,2006-09-12,2006-09-12,2006-09-12,2006-09-12,2006-09-13,AD


In [55]:
df_FDG_ROI_npiq_DX.loc[df_FDG_ROI_npiq_DX['DX']=='MCI to AD','DX']='AD'
df_FDG_ROI_npiq_DX.loc[df_FDG_ROI_npiq_DX['DX']=='NL to MCI','DX']='MCI'

In [58]:
df_FDG_ROI_npiq_DX[(df_FDG_ROI_npiq_DX['DX'].isin(['NL','MCI','AD'])) & (df_FDG_ROI_npiq_DX['NPID'].isin([0,1]))].groupby(['DX','NPID']).ROInorm.count()

DX   NPID
AD   0.0     313
     1.0     193
MCI  0.0     737
     1.0     222
NL   0.0     472
     1.0      43
Name: ROInorm, dtype: int64

In [59]:
df_FDG_ROI_npiq_DX[(df_FDG_ROI_npiq_DX['DX'].isin(['NL','MCI','AD'])) & (df_FDG_ROI_npiq_DX['NPID'].isin([0,1]))].groupby(['DX','NPID']).ROInorm.mean()

DX   NPID
AD   0.0     1.007670
     1.0     1.019543
MCI  0.0     1.178913
     1.0     1.170802
NL   0.0     1.263174
     1.0     1.183447
Name: ROInorm, dtype: float64